In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


# Aplicação dos modelos desenvolvidos para classificação das decisões judiciais

O processo de mineração de dados textuais de decisões judiciais elaborado resultou no desenvolvimento de dois modelos baseados em aprendizado de máquina supervisionado e na sua utilização para o processamento de mais de 20 mil decisões judiciais dos Tribunais do Trabalho da 3ª e 4ª Região. Com a utilização desses modelos, os objetivos de mineração foram atingidos que eram classificar automaticamente os acórdãos judiciais em relação ao deferimento ou não e classificar automaticamente o requerente do recurso em relação a ser empresa ou a ser empregado.

Desse modo, foi possível atender também ao objetivo de negócio que seria analisar as decisões judiciais por meio de algoritmo computacional para verificar se seria possível que os tribunais avaliados e suas Turmas Recursais julgue favoravelmente quantidade significativamente maiores de recursos para uma das partes do que para outra em média.

# Carregamento da base de dados a ser processada

In [2]:
import pandas as pd

arquivo = "..//3_Preparacao_dos_dados//3_1_Transformacao_dos_dados//Datasets_transformados_final//TRT3//TRT3_acordaos_amostra_2017_2018_2019_completo_com_indice_relator_limpo.csv"
df_trt3 = pd.read_csv( arquivo, encoding='utf-8' )
print( df_trt3.info() )

arquivo = "..//3_Preparacao_dos_dados//3_1_Transformacao_dos_dados//Datasets_transformados_final//TRT4//TRT4_acordaos_amostra_2017_2018_2019_completo_com_indice.csv"
df_trt4 = pd.read_csv( arquivo, encoding='utf-8' )
print( df_trt4.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10875 entries, 0 to 10874
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   INDEX            10875 non-null  int64  
 1   INTEIRO_TEOR     10875 non-null  object 
 2   DATA             10875 non-null  object 
 3   RELATOR          10875 non-null  object 
 4   ORGAO            10875 non-null  object 
 5   RECORRENTE       10875 non-null  object 
 6   DISPOSITIVO      10875 non-null  object 
 7   QTD_RECORRENTES  10875 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 679.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12071 entries, 0 to 12070
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   INDEX            12071 non-null  int64  
 1   LINK             12071 non-null  object 
 2   INTEIRO_TEOR     12071 non-null  object 
 3   DATA             120

In [3]:
df_trt3["TRIBUNAL"] = "TRT3"
print( df_trt3.info() )

df_trt4["TRIBUNAL"] = "TRT4"
print( df_trt4.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10875 entries, 0 to 10874
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   INDEX            10875 non-null  int64  
 1   INTEIRO_TEOR     10875 non-null  object 
 2   DATA             10875 non-null  object 
 3   RELATOR          10875 non-null  object 
 4   ORGAO            10875 non-null  object 
 5   RECORRENTE       10875 non-null  object 
 6   DISPOSITIVO      10875 non-null  object 
 7   QTD_RECORRENTES  10875 non-null  float64
 8   TRIBUNAL         10875 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 764.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12071 entries, 0 to 12070
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   INDEX            12071 non-null  int64  
 1   LINK             12071 non-null  object 
 2   INTEIRO_TEOR     12

In [4]:
df_tudo = df_trt4.copy()
df_tudo = df_tudo.append(df_trt3)
df_tudo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22946 entries, 0 to 10874
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   INDEX            22946 non-null  int64  
 1   LINK             12071 non-null  object 
 2   INTEIRO_TEOR     22946 non-null  object 
 3   DATA             22946 non-null  object 
 4   RELATOR          22946 non-null  object 
 5   ORGAO            22946 non-null  object 
 6   RECORRENTE       22946 non-null  object 
 7   DISPOSITIVO      22946 non-null  object 
 8   QTD_RECORRENTES  22946 non-null  float64
 9   TRIBUNAL         22946 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.9+ MB


In [5]:
df_tudo["QTD_RECORRENTES"].unique()

array([1.])

In [6]:
df_tudo["TRIBUNAL"].value_counts()

TRT4    12071
TRT3    10875
Name: TRIBUNAL, dtype: int64

In [7]:
df_tudo.sample(5)

,INDEX,LINK,INTEIRO_TEOR,DATA,RELATOR,ORGAO,RECORRENTE,DISPOSITIVO,QTD_RECORRENTES,TRIBUNAL
4411,4411,NaN,PJe: 0011202-22.2016.5.03.0028 (RO) Dispon...,31/10/2018,Gisele de Cassia VD Macedo,Oitava Turma,MANOEL DIAS DA ROCHA,Cabeçalho do acórdão Acórdão Fundamentos pelos...,1.0,TRT3
4274,4274,NaN,PJe: 0013010-59.2017.5.03.0050 (RO) Dispon...,26/06/2018,Danilo Siqueira de C.Faria,Terceira Turma,LYND CALÇADOS LTDA.,Cabeçalho do acórdão Acórdão ACORDAM os Desemb...,1.0,TRT3
5973,5973,NaN,PJe: 0011141-96.2014.5.03.0040 (RO) Dispon...,31/10/2018,Vitor Salino de Moura Eca,Setima Turma,MOZAR OZÓRIO DE PAULA,Cabeçalho do acórdão CONCLUSÃO Fundamentos pel...,1.0,TRT3
3795,3795,rest/cache/acordao/pje/p4UaytPaJzShvrGXWCkH9w?,Acórdão: 0020708-13.2015.5.04.0001 ( ROT )...,06/10/2017,CARLOS HENRIQUE SELBACH,2ª Turma,DANIELA CARVALHO DA SILVA,"Vistos, relatados e discutidos os autos. ACOR...",1.0,TRT4
4656,4656,rest/cache/acordao/pje/_Td3sG0hsYujsPloADZYSg?,Acórdão: 0020524-96.2017.5.04.0030 ( ROT )...,22/05/2019,LUIS CARLOS PINTO GASTAL,3ª Turma,KATIA MARIANTE GRANJA,"Vistos, relatados e discutidos os autos. ACOR...",1.0,TRT4


# Estrutura de dados para classificação do tipo de requerente

### Carregamento do modelo desenvolvido com o algoritmo Support Vector Machine (SVM)

In [8]:
%%time

from joblib import load
model_tipo_requerente = load('..//4_Modelagem//4_1_Classificador_tipo_recorrente//ml_model_recorrente_SVM.joblib') 

CPU times: user 752 ms, sys: 568 ms, total: 1.32 s
Wall time: 1.18 s


### Teste do modelo

In [9]:
EMPREGADO = 1
EMPRESA = 0

In [10]:
assert [EMPREGADO] in model_tipo_requerente.predict(["joao da silva"])

In [11]:
assert [EMPRESA] in model_tipo_requerente.predict(["empresa SA"])

# Estruturas de dados para classificação do deferimento ou não da decisão

## Tokenizador de palavras da lingua portuguesa

In [22]:
#python -m spacy download pt_core_news_sm
import spacy
from spacy.tokens import Doc
nlp = spacy.load("pt_core_news_sm")

from nltk.tokenize import RegexpTokenizer
def tokenize(text):
    """
    Tokenize the given sentence in Portuguese.
    :param text: text to be tokenized, as a string
    """
    tokenizer_regexp = r'''(?ux)
    # the order of the patterns is important!!
    # more structured patterns come first
    [a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+|    # emails
    (?:https?://)?\w{2,}(?:\.\w{2,})+(?:/\w+)*|                  # URLs
    (?:[\#@]\w+)|                     # Hashtags and twitter user names
    (?:[^\W\d_]\.)+|                  # one letter abbreviations, e.g. E.U.A.
    (?:[DSds][Rr][Aa]?)\.|            # common abbreviations such as dr., sr., sra., dra.
    (?:\B-)?\d+(?:[:.,]\d+)*(?:-?\w)*|
        # numbers in format 999.999.999,999, possibly followed by hyphen and alphanumerics
        # \B- avoids picks as F-14 as a negative number
    \.{3,}|                           # ellipsis or sequences of dots
    \w+|                              # alphanumerics
    -+|                               # any sequence of dashes
    \S                                # any non-space character
    '''
    tokenizer = RegexpTokenizer(tokenizer_regexp)

    return tokenizer.tokenize(text)

class PortugueseTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = tokenize( text )
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

nlp.tokenizer = PortugueseTokenizer(nlp.vocab)

In [23]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
nlp.Defaults.stop_words = stopwords[:100]

In [25]:
#codigo para tirar o "não" das stop words pq eu preciso dele nas funções de rotulação
if "não" in nlp.Defaults.stop_words:
    nlp.Defaults.stop_words.remove("não")
if "dar" in nlp.Defaults.stop_words:
    nlp.Defaults.stop_words.remove("dar")
nlp.Defaults.stop_words

['de',
 'a',
 'o',
 'que',
 'e',
 'é',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo']

In [26]:
#alterada esss funcao para tbm remover as stop words
def get_lemma_form( frase ):
    juntado = ""
    for t in nlp( str( frase ) ):
        if not t.is_stop: #remove stop words tbm
            juntado += t.lemma_ +" "
    return juntado.strip()

In [27]:
#esse assert da erro, por isso eu criei a funcao limpa() abaixo.
#assert "dar parcial provimento" in get_lemma_form( "deu-se parcial provimento")

In [28]:
#ajuda a remover traços do texto
def limpa( frase ):
    import re
    cleanString = re.sub('\W+',' ', frase )
    novo = ""
    for w in cleanString.split( " " ):
        pronomes_obliquos = [ "me", "te", "se", "lhe", "lhes", "nos", "vos" ]
        if w not in pronomes_obliquos:
            novo += " "+w
    return novo.strip()

In [29]:
def limpa_tudo(frase):
    return limpa( get_lemma_form( frase ) )

In [30]:
def tokenizer( texto ):
    # Lemmatizing each token and converting each token into lowercase
    texto = nlp( str( texto ) )
    return [ limpa_tudo( word.lemma_.lower().strip() ) if word.lemma_ != "-PRON-" else word.lower_ for word in texto ]

### Testes das funções de limpeza de texto

In [31]:
assert 'dar provimento' in limpa_tudo( "dar provimento" )

In [32]:
assert 'dar parcial provimento' in limpa_tudo("dar parcial provimento")

# Erro no assert
Se ocorrer erro no assert (provavelmente ocorrerá) é porque deu algum problema no carregamento das stopwords dentro do Spacy.
<br><strong>Solução: reexecutar as células a partir do título "Tokenizador de palavras da lingua portuguesa" que o carregamento funcionará sem nenhuma outra intervenção. Deve haver algum bug aí na biblioteca.</strong>

## Carregamento do Pipeline de vetorização dos documentos

In [33]:
%%time

from joblib import load
text_transform_pipeline_deferimento = load('..//4_Modelagem//4_2_Classificador_deferimento//4_2_2_Dataset_Weak_Supervision//4_2_2_1_Com_dataset_WS_Completo//ml_text_transform_pipeline_ws_completo.joblib') 

CPU times: user 179 ms, sys: 2.53 ms, total: 182 ms
Wall time: 298 ms


## Carregamento do modelo desenvolvido com o algoritmo Gradient Boosting Classifier

In [34]:
%%time

from joblib import load
model_deferimento = load('..//4_Modelagem//4_2_Classificador_deferimento//4_2_2_Dataset_Weak_Supervision//4_2_2_1_Com_dataset_WS_Completo//ml_model_deferimento_GradientBoostingClassifier.joblib') 

CPU times: user 53.4 ms, sys: 7.85 ms, total: 61.2 ms
Wall time: 159 ms


### Teste do modelo

In [35]:
INDEFERIMENTO = 0
DEFERIMENTO = 1
SEM_ANALISE_MERITO = 2

In [36]:
teste_dispositivo_deferimento = 'Vistos, relatados e discutidos os autos.  ACORDAM os Magistrados integrantes da 1ª Turma do Tribunal Regional do Trabalho da 4ª Região: por maioria de votos, vencidas parcialmente as Desembargadoras Relatora e Iris Lima de Moraes, com votos díspares, DAR PARCIAL PROVIMENTO AO RECURSO ORDINÁRIO DA RECLAMADA FRAS-LE S/A para [a]limitar a condenação ao pagamento de 20 minutos de horas in itinere quando a jornada se iniciava após às 23h45min e dez minutos por dia laborado pela troca de uniforme, com reflexos em repousos semanais remunerados e feriados, férias acrescidas de 1/3, 13º salário, no FGTS com a indenização de 40%; [b] excluir da condenação o pagamento de uma hora extra pela ausência de fruição integral do intervalo, pelo acréscimo do tempo de trajeto, de troca de uniforme, e de deslocamento dentro da empresa; [c] para excluir da apuração do adicional noturno as horas in itinere , o tempo de deslocamento e espera e o tempo de troca de uniforme; [d] para excluir da condenação o pagamento das horas in itinere . Valor da condenação reduzido para R$ 4.000,00, com custas proporcionais de R$ 80,00, pela reclamada.  Intime-se.  Porto Alegre, 31 de maio de 2017 (quarta-feira).          Cabeçalho do acórdão               Acórdão'
teste_dispositivo_deferimento_vetorizado = text_transform_pipeline_deferimento.transform([teste_dispositivo_deferimento])
assert DEFERIMENTO in model_deferimento.predict(teste_dispositivo_deferimento_vetorizado)

In [37]:
teste_dispositivo_indeferimento = 'Vistos, relatados e discutidos os autos.  ACORDAM os Magistrados integrantes da 5ª Turma do Tribunal Regional do Trabalho da 4ª Região: à unanimidade de votos, negar provimento ao recurso ordinário da reclamante.   Intime-se.  Porto Alegre, 18 de junho de 2019 (terça-feira).          Cabeçalho do acórdão               Acórdão'
teste_dispositivo_indeferimento_vetorizado = text_transform_pipeline_deferimento.transform([teste_dispositivo_indeferimento])
assert INDEFERIMENTO in model_deferimento.predict(teste_dispositivo_indeferimento_vetorizado)

In [38]:
teste_dispositivo_sem_analise_merito = "Sem valor de condenação. Prejudicada a análise do recurso ordinário das partes."
teste_dispositivo_sem_analise_merito_vetorizado = text_transform_pipeline_deferimento.transform([teste_dispositivo_sem_analise_merito])
assert SEM_ANALISE_MERITO in model_deferimento.predict(teste_dispositivo_sem_analise_merito_vetorizado)

## Vetorização dos documentos de texto das decisões

Primeira execução ou execução limpa: rodar as próximas duas celulas. Se quiserem abrir o array ja processado, executar a célula com o np.load()

In [44]:
%%time

dispositivo_vetorizado = text_transform_pipeline_deferimento.transform(df_tudo["DISPOSITIVO"].values)
dispositivo_vetorizado

CPU times: user 4h 39min 27s, sys: 3.17 s, total: 4h 39min 30s
Wall time: 4h 39min 27s


<22946x29087 sparse matrix of type '<class 'numpy.float64'>'
	with 1903346 stored elements in Compressed Sparse Row format>

In [45]:
import numpy as np
with open('dispositivo_vetorizado.np', 'wb') as f:
    np.save(f, dispositivo_vetorizado)

In [46]:
#import numpy as np

#dispositivo_vetorizado = np.load( "dispositivo_vetorizado.np", allow_pickle=True )

# Aplicação dos modelos

### Classificação do tipo de requerente

In [47]:
tipo_requerente_previsto = model_tipo_requerente.predict(df_tudo["RECORRENTE"].values)

In [48]:
df_tudo["tipo_requerente_previsto"] = tipo_requerente_previsto

### Classificação do deferimento ou não da decisão

In [49]:
deferimento_previsto = model_deferimento.predict(dispositivo_vetorizado)

In [50]:
df_tudo["deferimento_previsto"] = deferimento_previsto

In [51]:
df_tudo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22946 entries, 0 to 10874
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   INDEX                     22946 non-null  int64  
 1   LINK                      12071 non-null  object 
 2   INTEIRO_TEOR              22946 non-null  object 
 3   DATA                      22946 non-null  object 
 4   RELATOR                   22946 non-null  object 
 5   ORGAO                     22946 non-null  object 
 6   RECORRENTE                22946 non-null  object 
 7   DISPOSITIVO               22946 non-null  object 
 8   QTD_RECORRENTES           22946 non-null  float64
 9   TRIBUNAL                  22946 non-null  object 
 10  tipo_requerente_previsto  22946 non-null  int64  
 11  deferimento_previsto      22946 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.3+ MB


# Salvamento da base de dados final

In [52]:
df_tudo.to_csv("df_classificado.csv", encoding='utf-8', index=False )